## Six-bar Mechanism Balancing

In [1]:
from BetaShF import ShF
from BetaShM import ShM 
import numpy as np 
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt

### Contraints

$-0.16m <= x_{cn},y_{cn} <= 0.16m$

$0.005m <= t_{cn} <= 0.04m$

### Objective Function

In [2]:
def objective_function(s, ShF, ShM): #c is a constant that distributes the weight among the functions.
    c = 0.5
    return c*ShF(s) + c*ShM(s)

### Differential Evolution

In [10]:
# bounds = [ #x,y,t
#     [-0.16, 0.16], 
#     [-0.16, 0.16], 
#     [0.005, 0.04], 

#     [-0.16, 0.16], 
#     [-0.16, 0.16], 
#     [0.005, 0.04], 

#     [-0.16, 0.16], 
#     [-0.16, 0.16], 
#     [0.005, 0.04], 

#     [-0.16, 0.16], 
#     [-0.16, 0.16], 
#     [0.005, 0.04], 

#     [-0.16, 0.16], 
#     [-0.16, 0.16], 
#     [0.005, 0.04], 
# ]

nVar = 5
bounds = []
for i in range(1,nVar*3+1):
    if(i%3==0): bounds.append([0.005,0.04])
    else: bounds.append([-0.16, 0.16])
print(bounds)


[[-0.16, 0.16], [-0.16, 0.16], [0.005, 0.04], [-0.16, 0.16], [-0.16, 0.16], [0.005, 0.04], [-0.16, 0.16], [-0.16, 0.16], [0.005, 0.04], [-0.16, 0.16], [-0.16, 0.16], [0.005, 0.04], [-0.16, 0.16], [-0.16, 0.16], [0.005, 0.04]]


In [11]:
it = 10
n = 250
sols = np.zeros((n, 2))
best, bestSol = 10, None
for i in range(n):
    r = differential_evolution(objective_function, bounds, args=(ShF, ShM), maxiter=it)
    sols[i][0], sols[i][1] = ShF([*r['x']]), ShM([*r['x']])
    if r['fun'] < best:
        best = r['fun']
        bestSol = r

In [ ]:
plt.plot(sols[:, 0], sols[:, 1], 'o')
plt.plot(ShF(*bestSol['x']), ShM(*bestSol['x']), '^')

In [ ]:
np.sqrt(best)

In [ ]:
bestSol

### Gradient Descent

In [ ]:
def gradient_descent(max_iter):
    pass
